# MGT-499 Statistics and Data Science - Individual Assignment

In [ ]:
# Import here what you need

import os
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import skew

This notebook contains the individual assignment for the class MGT-499 Statistics and Data Science. Important information:
- **Content**: the assignment is divided in two main parts, namely data cleaning (2 datasets) and Exploratory Data Analysis, for a total of 13 main questions (see table of contents). Some of these main questions are divided in sub questions. In the first part, the questions are very specific, while in the second part they are more open.
- **Deadline**: Tuesday 8th of November at 23:59. 
- **Final Output**: a Jupyter notebook, which we (teachers) can run. 
- **Answering the Questions**: you will find the questions in markdown cells below. Under each of these cells, you will find a cell / cells for answers. Type there your answer. For the answer to be correct, the cell with the answer must run without error (unless specified). You can use markdown cells for the answers that require text.
- **Submission**: submit the assignment on Moodle, under [Individual Assignment](https://moodle.epfl.ch/mod/assign/view.php?id=1222846)

## Content
- [Polity5 Dataset](#polity5)  
    - [Question 1: Import the data and get a first glance](#question1)
    - [Question 2: Select some variables](#question2)
    - [Question 3: Missing Values](#question3)
    - [Question 4: Check Polity2](#question4)
- [Quality of Government (QOG) Environmental Indicators Dataset](#qog)  
    - [Question 5: Import the data and do few fixes](#question5)
    - [Question 6: Merge QOG and Polity5 ... first attempt](#question6)
    - [Question 7: Merge QOG and Polity5 ... second attempt](#question7)
    - [Question 8: Clean the merged dataframe](#question8)
- [Exploratory Data Analysis](#eda)
    - [Question 9: Selecting the ingredients for the recipe (how I select the variables)](#question9)  
    - [Question 10: Picking the right quantity of each ingredient (how I select my sample)](#question10)
    - [Question 11: Tasting and preparing the ingredients (univariate analysis)](#question11)
    - [Question 12: Cooking the ingredients together (bivariate analysis)](#question12)
    - [Question 13: Tasting the new recipe (conclusion)](#question13)

## Polity5 data <a class="anchor" id="polity5"></a>

Polity5 is a widely used democracy scale. The raw data as well as the codebook are available [here](http://www.systemicpeace.org/inscrdata.html). For this assignment, we have modified a bit the original version, for example we have added the iso3 code for countries to make you save time. You can find the modified version [here](https://raw.githubusercontent.com/edoardochiarotti/class_datascience/main/Notebooks/Assignment/individual_assignment/data/polity2_iso3.csv).

### Question 1: import the data and get a first glance <a class="anchor" id="question1"></a>

1a) Import the csv 'polity2_iso3.csv' (file provided in the link [here](https://raw.githubusercontent.com/edoardochiarotti/class_datascience/main/Notebooks/Assignment/individual_assignment/data/polity2_iso3.csv)) as a panda dataframe (ignore the warning message) **(1 point)**

In [ ]:
# Answer 1a
url = 'https://raw.githubusercontent.com/edoardochiarotti/class_datascience/main/Notebooks/Assignment/individual_assignment/data/polity2_iso3.csv'
polity_data = pd.read_csv(url, low_memory=False)

1b) Display the first 10 rows **(1 point)**

In [ ]:
# Answer 1b
polity_data.head(10)

1c) Display the data types of all the variables included in the data **(1 point)**

In [ ]:
# Answer 1c
polity_data.dtypes

1d) By looking at your answer in 1c, what is the difference between the different types of variables? Why the type of some variables is defined as object? **(1 point)**

Answer 1d:

We can see that we have four different dataypes: `object`, `float64` and `int64`. `Object` is  the pandas type for mixed values s.a. strings and numbers. In python the equivivalent  type is `string`. `Float64` refers to numeric characters with decimals. If columns contain numbers and NaN pandas will defualt to `float64`. And for `int64` is for integer numbers, and the 64 refers to memory allocation.

### Question 2. Select some variables <a class="anchor" id="question2"></a>

2a) Create a subset dataframe that contains the variables 'iso3', 'country', 'year', 'polity2' and display it **(1 point)**

In [ ]:
# Answer 2a
polity_data_subset = (polity_data[['iso3', 'country', 'year', 'polity2']]).copy()
polity_data_subset

2b) Display the type of the variable "year" **(1 point)**

In [ ]:
# Answer 2b
polity_data_subset['year'].dtypes

2c) Convert the variable "year" to string **(1 point)**
<br>
Hint: if you get a warning message of the type "SettingWithCopyWarning", it is because you did not subset the data in the right way. Go back to your class notes and check the different ways to subset a dataframe, and try again. If you do it correctly, you will not get the warning message.

In [ ]:
# Answer 2c
polity_data_subset['year'] = polity_data_subset['year'].astype('string')
polity_data_subset['year'].dtypes

### Question 3: Missing Values <a class="anchor" id="question3"></a>

3a) Subset the rows that have iso3 missing and display **(1 point)**

In [ ]:
# Answer 3a
iso_missing = polity_data_subset[polity_data_subset['iso3'].isna()]
iso_missing


3b) Display the countries that have missing iso3. What can you tell by looking at them? Any similarities? **(1 point)**

In [ ]:
# Answer 3b
iso_missing_contries = iso_missing['country']
print(f'They seem to be old countries, or old country names.\n\n{iso_missing_contries.unique()}')


3c) Display the countries with missing iso3 from 2011. **(1 point)**

In [ ]:
# Answer 3c
# Interpreting the question so that we take the contries missing iso3 from 2011 onwards
global df_missing_iso3_since_2011
global df_missing_iso3_since_2011_list

df_missing_iso3_since_2011 = iso_missing[iso_missing['year'].astype(int) >= 2011]
df_missing_iso3_since_2011_list = sorted(df_missing_iso3_since_2011['country'].unique())
df_missing_iso3_since_2011_list

3d) Display the rows for which the column "country" contains the word "Serbia". By looking at the result, can you tell what happened to Serbia in 2006? **(1 point)**
<br>
Hint: the most general way of doing this is to use a combination of re.search and list comprehension. To display the full subset, you can use print(df.to_string()).

In [ ]:
# Answer 3d
polity_data_subset_serbia = polity_data_subset[polity_data_subset['country'].str.contains(pat='Serbia', case=False)]
print(polity_data_subset_serbia.to_string())

We can see that the country referred to as Serbia, reappeared in 2006, 
This is due to a name change following Montenegro and Serbias decleration of independence of their previous union named `Serbia and Montenegro`.


3e) Write a function that does the operation in 3d and use it to display the subset that has the word "sudan" (all lower cap) in country. Then do the same for the word "vietnam" (all lower cap). **(1 point)**
<br>
Hint: options of functions can be very useful.

In [ ]:
# Answer 3e

# Function find_country
# input: dataframe, column, name
# output: df containting the specified string in the column 'country'
# making sure that the type is correct, helping me later if use the wrong type

def find_rows_with_pattern(df:pd.DataFrame, column:str, name:str):
	try:
		return df[df[column].str.contains(pat=name, case=False)]
	except TypeError as e:
		print(f'Not a valid datatype: {e}.\nUse {pd.DataFrame} and {str} as option type parameters')

df_sudan = find_rows_with_pattern(polity_data_subset, 'country', 'sudan')
df_vietnam = find_rows_with_pattern(polity_data_subset, 'country', 'vietnam')

print(df_sudan.to_string())
print(df_vietnam)

3f) Replace nan values in iso3 with correct iso3 for the 5 countries found in 3c from 2011 onwards, and display the subset with the fixed values to check that everything worked. **(1 point)**
<br>
Hint: the correct iso3 for these 5 countries are "ETH","MNE","SRB","SDN","VNM".

In [ ]:
# Answer 3f

# Storing the correct iso3 values in a list
iso3_code_missing = ('ETH', 'MNE', 'SRB', 'SDN', 'VNM')

# Creating a dict to look up correct iso3 values with list from 3c
iso3_dict = dict(zip(df_missing_iso3_since_2011_list, iso3_code_missing))

# Copying the politydata to work on
polity_data_subset_iso_replace = polity_data_subset.copy()

# Function: replace(df, dct, replaced)
# input: df: dataframe, dct: dictionary to loop through, replaced: item to be replaced
# manipulates in place

def replace(df, dct, replaced, from_year):
    for key, item in dct.items():
        df.loc[(df['country'] == key) & (df['year'].astype(int) >= from_year)] = df.loc[(
            df['country'] == key) & (df['year'].astype(int) >= 2011)].replace(replaced, item)


# Calling the function and telling which value i want to replace
replace(polity_data_subset_iso_replace, iso3_dict, np.nan, 2011)

# Using the indexes of the subset created in 3c to check if we have the correct countrycodes
polity_data_subset_iso_replace.iloc[df_missing_iso3_since_2011.index]



3g) Drop the remaining rows which have nan in "iso3' and display the new number of rows of the dataframe (how many are they?) **(1 point)**

In [ ]:
# Answer 3g

# Dropping NaNs in the iso3 column
polity_data_ss_cleaner = polity_data_subset_iso_replace.dropna(subset='iso3')

# Number of rows, and making sure that the data is consistent
print(f'Before removing rows with NaNs in column "iso3" we had: {len(polity_data_subset_iso_replace)} rows')
print(f'Now we have: {len(polity_data_ss_cleaner)} rows')
print(f'Which means that we removed: {len(polity_data_subset_iso_replace) - len(polity_data_ss_cleaner)} rows')
print(f'Which makes sense since we had {polity_data_subset_iso_replace["iso3"].isna().sum()} instances of NaNs in the polity2 column.')

### Question 4: Check Polity2 <a class="anchor" id="question4"></a>

4a) Display the first and last year included in the dataset **(1 point)**

In [ ]:
# Answer 4a

# Assuming the question is interpreted so that we are to find earliest and latest year recorded we have:
print(f'Earliest entry year: {polity_data["year"].min()}, Latest entry year: {polity_data["year"].max()}')

# Assuming the assignment is asking for the first and last year in the first and last row of the original dataset
print(f'First row year: {polity_data.iloc[[0, -1]]["year"].values[0]}, and last row year: {polity_data.iloc[[0, -1]]["year"].values[-1]}')


4b) What do the values in "polity2" represent? **(1 point)**

Answer 4b: 

`Polity2` is a revised version of the Polity score, which captures a regime authority on a `21-point` scale ranging from `-10` (hereditary monarchy) to `10` (consolidated democracy). `-66` are cases of foreign interruption and are treated as missing values. Whearas `-88` represents transitions. If a given country has the value of `-5` i 1990 and `+5` in 2000, it means that it has an annual increase of `+1`; and the converted scores are `1991: -4 1992: -3 ... 2000: +5` 

The documentation I used is [here](https://www.systemicpeace.org/inscr/p4manualv2016.pdf).

4c) Do we have weird values for polity2? If yes, why? What should we do about them? Transform the data accordingly. **(1 point)**

Answer 4c:

In [ ]:
# Answer 4c

# We can see that we have NaNs in polity2
print(f'Number of NaNs in the df per column: \n {polity_data_ss_cleaner.isna().sum()}')
print(f'\nThe unique values of polity2: \n {polity_data_ss_cleaner.polity2.unique()}')

# To repeat:
# -66 : cases of foreign “interruption” are treated as “system missing.”
# -88 : Cases of “transition” are prorated across the span of the transition. 
# So the NaNs may be there for a reason...

# Creating a copy df to work on
df_cleaned = polity_data_ss_cleaner.copy()

# Lets find the rows with -66 and -88
polity_66_88_index = df_cleaned[(df_cleaned.polity2 == -66) | (df_cleaned.polity2 == -88)].index
print(f'\nRows with -66 and -88:\n{polity_data_subset.iloc[polity_66_88_index].to_string()}')

# In addition -88 values should be treated as missing values, as i have looked into the data for other occupied countries during the world wars
# We see that they have NaNs and not -66. I will therefor change the -66 to NaN
df_cleaned.loc[df_cleaned.polity2 == -66] = df_cleaned.loc[(df_cleaned.polity2 == -66)].replace(-66, 0)

# As Belgium has -88 for the first row, and all the nearest following years has score -4 i will change this value to -4
df_cleaned.loc[df_cleaned.polity2 == -88] = df_cleaned.loc[(df_cleaned.polity2 == -88)].replace(-88, np.float64(-4))

# Printing the current unique polity2 values to verify changes
print(f'\nThe new unique values of polity2: \n {df_cleaned.polity2.unique()}')


4d) Make a map that shows the number of observations of polity2 by country **(1 point)**

In [ ]:
# Answer 4d

pol_obs_per_country = df_cleaned.groupby(['iso3'])['polity2'].size().reset_index()
pol_obs_map = px.choropleth(pol_obs_per_country, locations='iso3',
                    title='Number of polity2 observations by country',
                    locationmode='ISO-3',
                    color='polity2', 
                    hover_name='iso3',
                    color_continuous_scale=px.colors.sequential.Plasma)

pol_obs_map.show()

4e) Store the final dataframe (the one you obtained after 5d) in an object called df_pol **(1 point)**

In [ ]:
# Answer 4e
df_pol = df_cleaned

## Quality of Government Environmental Indicators <a class="anchor" id="qog"></a>

The QoG Environmental Indicators dataset (QoG-EI) (Povitkina, Marina, Natalia Alvarado Pachon & Cem Mert Dalli. 2021). The Quality of Government Environmental Indicators Dataset, version Sep21. University of Gothenburg: The Quality of Government Institute, https://www.gu.se/en/quality-government), is a compilation of indicators measuring countries' environmental performance over time, including the presence and stringency of environmental policies, environmental outcomes (emissions, deforestation, etc.), and public opinion on the environment. Codebook and data are available [here](https://www.gu.se/en/quality-government/qog-data/data-downloads/environmental-indicators-dataset).

### Question 5: Import the data and do few fixes <a class="anchor" id="question5"></a>

5a) Import data from the Quality of Government Environmental Indicators Dataset and display the variables types and the number of rows **(1 point)**
<br>
Hint: When you go on the webpage of the Environmental Indicators Dataset, you can directly import from a URL by copying the link address of the dataset! 

In [ ]:
# Answer 5a
url = 'https://www.qogdata.pol.gu.se/data/qog_ei_sept21.csv'
df_qog = pd.read_csv(url, encoding='latin-1')

print(f'Variable types: {df_qog.dtypes} \n\n Number of rows: {len(df_qog)}')

5b) Rename the variable "ccodealp" to "iso3" **(1 point)**

In [ ]:
# Answer 5b
df_qog.rename(columns={'ccodealp': 'iso3'}, inplace=True)

5c) Check the type of the variables "year" and "iso3" are string, if not convert them to string **(1 point)**

In [ ]:
# Answer 5c
print(f'Before convert \nYear datatype: {df_qog.year.dtypes}')
print(f'iso3 datatype: {df_qog.iso3.dtypes}')

df_qog[['year', 'iso3']] = df_qog[['year', 'iso3']].astype('string')

print(f'After convert \nYear datatype: {df_qog.year.dtypes}')
print(f'iso3 datatype: {df_qog.iso3.dtypes}')

### Question 6: Merge QOG and Polity5 ... issues with QOG? <a class="anchor" id="question6"></a>

6a) Get a subset of the dataframe that includes the variables "cname", "iso3", "year" and "cckp_temp", and display the number of rows. **(1 point)**

In [ ]:
# Answer 6a

df_qog_subset = df_qog[['cname', 'iso3',  'year', 'cckp_temp']]
len(df_qog_subset)

6b) Merge this subset (left) and the clean version of the polity data (right), using the argument how="left". Was the merge succesfull? If yes, how many rows has the merged dataframe? Is it the same number of rows of the subset in 6a? **(1 point)**

In [ ]:
# Answer 6b

merged = pd.merge(left=df_qog_subset, right=df_pol, how='left')
print(f'Yes - With {len(merged)} rows, its the same amount of rows at we saw in 6a')
merged

6c) Do the same by adding the argument validate="one-to-one". Can you make some hypotheses on why you get an error? **(1 point)**

In [ ]:
# Answer 6c
merge_one_to_one = pd.merge(left=df_qog_subset, right=df_pol, validate='1:1')


By using the option validate='one-to-one', we get an error message which states that the merge keys are not unique. 
Which makes sense since we for example have several rows in the same column with the same value.There may be ambiguity in the keys in which the merge uses to match the two datasets.

6d) Consider the subset of the QOG you obtained in 6a and write a code to (i) count the number of observations for the variable "cckp_temp" for each combination of iso3 and year, (ii) store the results in a dataframe. For example, the combination "USA-2012" should have 1 observation for "cckp_temp", so the result of your code should be 1. The code should do this for all iso3-year combinations of your subset dataframe, and store the results in a dataframe. **(1 point)**
<br>
Hint: it should not take you more than 2 lines of code.

In [ ]:
# Answer 6d
cckp_temp_per_combination = df_qog_subset.groupby(['iso3', 'year'])['cckp_temp'].size().reset_index()
cckp_temp_per_combination


6e) Use the code in 6d to write a function that displays all rows of the dataframe obtained in 6a that have more than one observation of "cckp_temp" for each iso3-year combination, and check if it works. **(1 point)**

In [ ]:
# Answer 6e

def more_than_one_combination(df, c1, c2, obs):
	df = df.groupby([c1, c2])[obs].size().reset_index()
	df = df[df[obs].astype(int) > 1]
	return df

In [ ]:
combination1 = 'iso3'
combination2 = 'year'
observation = 'cckp_temp'

more_than_one_combination(df_qog_subset, combination1, combination2, observation)


6f) Which countries have more than one observation for each iso3-year combination? Deal with these countries in the subset dataframe created in 6a to make sure you no longer have double observations for iso3-year combinations, and check that after your fix this is actually the case. **(1 point)**
<br>
Hint: should we keep a country with all missing values?

In [ ]:
# Answer 6f

# Using the previous function to get the countries with more than one iso3-year combination
countries = more_than_one_combination(df_qog_subset, 'iso3', 'year', 'cckp_temp')

# Getting the iso3 code for that particualar country
iso3_duplicate_country = countries.iso3.unique()[0]

# Creating a df to show all the data with the iso3-year combiantion constraints 
iso3_duplicate_df = df_qog_subset[df_qog_subset.iso3 == iso3_duplicate_country]

# Inspecting the values, and seeing that all cckp_temp values for North Vietnam are NaN
print(iso3_duplicate_df.to_string())

# We saw that we only have NaN values for North Vietnam - so we lets find the indexes of these rows
# In addiiton the Polity5 dont have any country names North Vietnam so i think we can remove these
north_vietnam_indexes = df_qog_subset[df_qog_subset.cname == 'North Vietnam'].index

# Based on the indexes we can drop these rows
df_qog_cleaned = df_qog_subset.drop(index=north_vietnam_indexes)

# Now we can check whether we have any duplicated by running the function from 6e again.
more_than_one_combination(df_qog_cleaned,'iso3', 'year', 'cckp_temp').empty

We saw that we only have NaN values for North Vietnam, In addiiton the Polity5 dont have any country names North Vietnam that is why i dropped these rows.

6g) If your check went well, now you can perform the same operation directly in the QOG dataframe (not in the substed dataframe created in 6a). How many rows does now the QOG dataframe has? **(1 point)**

In [ ]:
# Answer 6g

# Using the function from 6e with the original dataset
more_than_one_iso_year_combination_df = more_than_one_combination(df_qog, 'iso3', 'year', 'cckp_temp')

# By inspecting the df it looks like we the same countries here. North Vietnam and Viet Nam.
print(more_than_one_iso_year_combination_df.to_string())

# Lets again find the indexes
north_vietnam_indexes = df_qog[df_qog.cname == 'North Vietnam'].index

# Dropping the North Vietnam indexes
df_qog = df_qog.drop(index=north_vietnam_indexes)

print(f'New length: {len(df_qog)}')

# We can now see that we dont have any multiple combinations of iso3 and year anymore
more_than_one_combination(df_qog, 'iso3', 'year', 'cckp_temp').empty

### Question 7: Merge QOG and Polity5 ... issues with Polity5? <a class="anchor" id="question7"></a>

7a) Merge the cleaned QOG dataframe (left) and the Polity dataframe (right) using the options how="left" and validate="one_to_one". Does it work? Why? **(1 point)**

In [ ]:
# Answer 7a
merge_qog_polity5 = pd.merge(left=df_qog, right=df_pol, how='left', validate='one_to_one')

# We get the error message that the keys are not unique in the right dataset

7b) Use the function you wrote in 6e to check what's wrong in the "clean" version of Polity **(1 point)**

In [ ]:
# Answer 7b

# By displaying the df_pol with iso3 value MNE we see that Kosovo has the wrong iso3 value. 
# This makes sense regarding the error message we got in 7a
polity_combination_issues_polity = more_than_one_combination(df_pol, 'iso3', 'year', 'polity2')
polity_combination_issues_country = more_than_one_combination(df_pol, 'iso3', 'year', 'country')
print(f"Checking which iso3-year combinations has several observations of polity2: \n {polity_combination_issues_polity}")
print(f"Checking which iso3-year combinations has several observations of countries: \n {polity_combination_issues_country}")
print(f'Checking the countries with iso3 code MNE: {df_pol[df_pol.iso3 == "MNE"]}')


7c) Drop or fix the countries that create troubles directly in the "clean" version of Polity and motivate your choices. **(1 point)**

In [ ]:
# Answer 7c
# Kosovo does not officially have a iso3 value but we can use XKX set by the European Commission
# OR we can choose to drop it completely as the QoG dataset does not include data on Kosova at all
# And since we only have values from 2011-2018 for Sudan-North and in QoG dataset dont have any
# values for Sudan-North/North-Sudan/Sudan North/North Sudan at all, we should drop these.

print(f'Checking if there are severeal countries with iso3 code SDN in QoG: {df_qog[df_qog.iso3 == "SDN"].cname.unique()}')
print(f"Checking if there is any data on Sudan-North in QoG: {find_rows_with_pattern(df_qog, 'cname', 'sudan-north').cname.unique()}")
print(f"Checking if there is any data on North-Sudan in QoG: {find_rows_with_pattern(df_qog, 'cname', 'north-sudan').cname.unique()}")
print(f"Checking if there is any data on North Sudan in QoG: {find_rows_with_pattern(df_qog, 'cname', 'sudan north').cname.unique()}")
print(f"Checking if there is any data on Sudan North  in QoG: {find_rows_with_pattern(df_qog, 'cname', 'north sudan').cname.unique()}")

# I have devided to drop Kosovo and Sudan-North
# Lets find the indexes of these rows
kosovo_indexes = df_pol[df_pol.country == 'Kosovo'].index
sudan_north_indexes = df_pol[df_pol.country == 'Sudan-North'].index

In [ ]:

# Based on the indexes we can drop these rows
df_pol = df_pol.drop(index=kosovo_indexes)
df_pol = df_pol.drop(index=sudan_north_indexes)

7d) Try now to merge the "clean-clean" versions of qog and Polity (the ones you obtained in 7g and 8c) always using the options how="left" and validate="one_to_one". Does it work, and why? How many rows has the resulting merged dataframe? **(1 point)**

In [ ]:
# Answer 7d
merge_qog_polity5 = pd.merge(left=df_qog, right=df_pol, how='left', on=['iso3', 'year'], validate='one_to_one')
len(merge_qog_polity5)

# It works as there is no longer any ambiguioty with the combination of iso3 and year in either of the dataframes.


### Question 8: Clean the merged dataframe <a class="anchor" id="question8"></a>

8a) In the merged dataframe, order the columns so that you have the "index" variables first and the variables with actual values last. **(1 point)**
<br>
Hint: index variables are "iso3", "year" and other similar variables you can find, and the variables with actual values are "polity2", "cckp_temp" and other similar variables you can find.

In [ ]:
# Answer 8a

# Displaying all the column names to get an overview
column_list = merge_qog_polity5.columns.tolist()
#print(column_list)

# Popping the columns we want to re-order
iso3_column = merge_qog_polity5.pop('iso3')
year_column = merge_qog_polity5.pop('year')
year_country = merge_qog_polity5.pop('country')

# Inserting columns in the specified order
merge_qog_polity5.insert(2, 'iso3', iso3_column)
merge_qog_polity5.insert(3, 'year', year_column)
merge_qog_polity5.insert(4, 'country', year_country)

column_list_after = merge_qog_polity5.columns.tolist()
#print(column_list_after)


8b) Rename "cname" as "country" and "country" as "country_polity". **(1 point)**

In [ ]:
# Answer 8b
merge_qog_polity5.rename(columns={'cname': 'country', 'country': 'country_polity'}, inplace=True)
merge_qog_polity5

8c) Save the clean merged dataframe as a csv in a subfolder called "clean_data" in your working directory **(1 point)**

In [ ]:
# Answer 8c

# Removing index columns
#merge_qog_polity5 = merge_qog_polity5.pop('Unnamed: 0')

# Cleaning path if exists
if os.path.exists('clean_data'):
	os.remove('clean_data/clean_data.csv')
	os.rmdir('clean_data')

# Creating folder and exporting csv
os.makedirs('clean_data')
merge_qog_polity5.to_csv('clean_data/clean_data.csv', index=False)

# I also want to add that i have done a lot of comparison check between the two csv files.
# I learnt that using the .eq or .equeal method on float number does not work very well,
# as these numbers are "floating", 1.0005 may in fact be 1.00005000102. 
# So i used np.isclose(a, b) - which works between a certain very small range.count
# I'm happy to say that the two datasets looks identical.

## Exploratory Data Analysis <a class="anchor" id="eda"></a>

In this section you will define a research question and perform a preliminary Exploratory Data Analysis (EDA) to address - or better, start addressing - the question at hand. This exercise will be done along the lines of the analysis done by our own Quentin Gallea in "*A recipe to empirically answer any question quickly*" ([Towards Data Science, 2022](https://towardsdatascience.com/a-recipe-to-empirically-answer-any-question-quickly-22e48c867dd5)). In this article, Quentin shows the first steps of an EDA that aims to explore whether heat waves have pushed governments to implement regulations against climate change (causal link). The logic is that, as it gets hotter and hotter, governments become more aware of climate change, and the problems it can cause to society, and start addressing it. In Quentin's analysis, heat waves (proxied by temperature) is the "main explanatory variable", rainfall is the "explanatory variable for heterogeneity", and regulations against climate change (proxied by the Environmental Policy Stringency Index) is the "outcome variable". He finds that indeed countries with relatively high temperatures have implemented more regulations against climate change. This is true especially when rainfall levels are low, as when it does not rain the damage of extreme heat is more evident to legislators, who therefore apply stricter regulations against these phenomenons.
<br>
<br>
In this exercise, you will be asked to do a similar analysis on a research question of your choice, using at least two of the variables of the dataset we have created in the former questions (QOG + Polity). For example, "what is the average temperature in 2010?" is not a valid research question (univariate), while "what is the impact of high temperatures on the stringency of climate regulations?" is a valid research question (at least bivariate). As before, we will ask you some (this time more general and open) questions, and you should report your answer in the cells below each question. Use a mix of markdown and code cells to answer (markdown for text and code for graphs and tables). We should be able to run all the graphs, i.e. screenshots of graphs are not accepted. Note that for now we have put only one markdown cell and one code cell for the answer, but feel free to add as many cells as you need.
<br>
Beyond the python code, we will grade the interpretations of the results and the coding decision you make.
<br>
<br>
Let your creativity guide you and let's have some fun!

### Question 9: Selecting the ingredients (how I select the variables) <a class="anchor" id="question9"></a>
We have saved the clean merged data that resulted from the previous questions in "clean_data_prepared_EDA" (it should be the same of the one you saved in "clean_data"). Import the clean merged data from "clean_data_prepared_EDA" using this [link](https://raw.githubusercontent.com/edoardochiarotti/class_datascience/main/Notebooks/Assignment/individual_assignment/clean_data_prepared_EDA/df_qog_polity_merged.csv). Explore the variables in the newly obtained dataframe by checking the documentation of QOG and Polity. Then, define a research question that addresses a causal link between at least two of these variables. Describe the research question, why you are addressing it and the variables of interest (outcome variable, main explanatory variable and explanatory variable for heterogeneity). **(3 points)**

Answer 9:

#### Climate Responsibility

Going back in time, from the start of the industrial revolution to now, highly industrialized countries have benefited greatly from polluting without consequence. High emission industries has created powerful nations with huge economies. So, if we aggregate total CO2 emission per capita per country, do the high polluters take responsibility for their historic and ongoing climate impact? I want to investigate the relationship between total CO2 emission per capita and degree of environmental action. Are the high-polluting countries paying the price or are they still exploiting their influence and power to further enhance their dominance?

Furthermore, one could argue that a country’s motivation to incorporate environment legislation comes from selfishness. Let me clarify with an imaginative example: Let's say a highly industrialized successful country faces a lot of climate threats such as extreme weather, wildfires, and flooding. Then it is in the country’s best interest to drive environmental politics. I would in this regard call this national climate responsibility, but not global per se. So to investigate this problem I will look at how the Climate Change Issue within each country effects the plot. The Climate Change Issue addresses to which extent each country faces climate change challenges.

As this is a personal question of mine, I’m very curious to see if I can find any correlation, causality, and or discover other underlying driving factors. I initially wanted to use the “Environmental Policy Performance Index” but as the country availability is almost limited to the the western countries, and to withstand the use of the same variable as Quentin I will use the “Climate change related tax revenue given in % of GDP" as my outcome variable. I find this suiting as it in this case directly translates to “paying for wrongdoing”.

So, to sum up:

- Outcome: Climate change related tax revenue (% of GDP) (`oecd_cctr_gdp`) *<cite>[1][1]</cite>
- Explanatory variable: CO2 emissions per capita (`edgar_co2pc`) *<cite>[2][2]</cite>
- Additional explanatory variable: Climate Change Issue Category (`epi_cch`) *<cite>[3][3]</cite>

[1]: oe.cd/pine
[2]: https://www.sgi-network.org/2020/Downloads
[3]: https://epi.envirocenter.yale.edu/



### Question 10: Picking the right quantity of each ingredient (how I select my sample) <a class="anchor" id="question10"></a>
Explore the data availability of your variables of interest and select a clean sample for the analysis. Describe this sample with the help of summary-statistics tables and maps. **(3 points)**

Answer 10:

The table below reveals that we have 180 observations for `epi_cch` - which is consistent with what the documentation said. It's an assessment of 8 indicators done in 2019 measuring climate challenge. The score varies from 0 to 100. As this i a new variable, it may not express the historic climate challenge a country faces. But as climate challenges are somewhat consistent i think that a country facing a lot of threats in 2019 may also have been so historically. So i will continue with this variable.

In [ ]:
url = 'https://raw.githubusercontent.com/edoardochiarotti/class_datascience/main/Notebooks/Assignment/individual_assignment/clean_data_prepared_EDA/df_qog_polity_merged.csv'
df_qog_polity_merged = pd.read_csv(url)

In [ ]:
# Answer 10:
# Subsetting and picking the variables i want to investigate
df_subset = df_qog_polity_merged[['country', 'iso3', 'year', 'oecd_cctr_gdp', 'edgar_co2pc', 'epi_cch']]
df_subset.describe()

In [ ]:
# Here we see all notna values for epi_cch - consistent with the documentation - all from 2019.
df_subset[df_subset.epi_cch.notna()]

Now lets see how dense our `Tax%GDP` data is. Between 2000 and 2017 we have mean above 80 - meaning we have Carbon Tax % of GDP for over 80 countries. This is quite good. I will use this time interval in my studies. Since climate change and its reactive measures did not get any widespread attention before Al Gore addressed GHG and its alarming warming effects in early 1990s, i will also neglect the effects on not using any data prior to 2000. I'm making an assumption that the  climate change tax as a percentage of GDP in % was relatively small prior to 1990s.

In [ ]:
df_subset[pd.notnull(df_subset['oecd_cctr_gdp'])].groupby(['year']).size().reset_index(name='counts')

In [ ]:
# But since we only have data for epi_cch from 2019. I will take this data out of the dataframe, remove all NaNs and then we can merge later on when we have aggregated the other datasets.

climate_challenge_df = df_subset[['iso3', 'year' , 'country', 'epi_cch']]

climate_challenge_clean = climate_challenge_df.dropna()

climate_challage_iso3_codes = climate_challenge_clean.iso3.unique()

# These are the countries that has a value for epi_cch (climate challenge issue) from 2019 and the countries we will work with.
climate_challage_iso3_codes


In [ ]:
# And for the time being we can pop the epi_cch column from our QoG_polity_subset
df_subset.pop('epi_cch')

Here is a table for the CO2 emission per capita for each country. The first recordings are from 1970. Which can make my hypothesis a bit hard to answer, as emissions before 1970 are not in this dataset. I tried looking online to see any datasets to merge in, and use. I found a reliable source at https://ourworldindata.org/co2-and-other-greenhouse-gas-emissions. My plan now is to use these values to give a more historically correct picture. The CO2 data has a corresponding GitHub page here: https://github.com/owid/co2-data.



In [ ]:
df_subset[pd.notnull(df_subset['edgar_co2pc'])].groupby(['year']).size().reset_index(name='counts')

### Fetching new data...

In [ ]:
# Fetching data from Our World In Data.
# This set is one of the most comprehensive data collections of total historically CO2 emissions.
url = 'https://nyc3.digitaloceanspaces.com/owid-public/data/co2/owid-co2-data.csv'
df_co2 = pd.read_csv(url)


Doing some housekeeping (subsetting a useable dataframe and changing iso_code to iso3)

In [ ]:
df_co2_subset = df_co2[['country', 'year', 'iso_code', 'population', 'co2']].copy()

df_co2_subset.rename(columns={'iso_code': 'iso3'}, inplace=True)

# Extracting the countries we have from climate_challage_iso3_codes.
df_co2_subset_country_limited = df_co2_subset[df_co2_subset.iso3.isin(climate_challage_iso3_codes)].copy()
len(df_co2_subset_country_limited)


In [ ]:
# Adding a column with co2 per capita and converting the scale so we have co2_per_capita and total co2 in tons
df_co2_subset_country_limited['co2'] = df_co2_subset_country_limited['co2']*1000000
df_co2_subset_country_limited['co2_per_capita'] =  df_co2_subset_country_limited['co2']/df_co2_subset_country_limited['population']

# And giving it a cleaner name...
df_co2_final = df_co2_subset_country_limited.copy()
df_co2_final = df_co2_final.dropna()

df_co2_final

Now i think we have a good working foundation for our emission data. 
### Moving to the QoG and Polity Data

Lets work further with our qog_polity dataset. I want to extract the data from 2000-2018. And then sum up the mean Climate change related tax revenue (% of GDP) per country.

In [ ]:
df_subset_2000_2018 = df_subset[(df_subset.year >= 2000) & (df_subset.year <= 2018)]

# Dropping the lackful emission column
df_subset_2000_2018_slim = df_subset_2000_2018.drop(columns='edgar_co2pc')

In [ ]:
# Lets also here limit the countries to the ones we found in climate_challage_iso3_codes

df_qog_pol_clean = df_subset_2000_2018_slim[df_subset_2000_2018_slim.iso3.isin(climate_challage_iso3_codes)].copy()
df_qog_pol_clean.describe()

# Wait .... hmmm - the mean for Tax to climate change as in % of GDP is 76%?


#### A mistake?
...Wait a minute, how can the `% of GDP` from Climate Change tax be over 100% and have an mean of 76%? This seems off. I will check the definition for the variable once again.

> Climate change-related tax revenue as a percentage of gross domestic product (GDP). Includes
taxes, fees and charges, tradable permits, deposit-refund systems, subsidies, and voluntary approaches related to the domain of climate change.


In my view this should probably be lower...

I will go to the data source <cite>[OECD][1]</cite> and see if the values there are the same. I found this <cite>[Compare Your Country (OECD data)][1]</cite> a simple graph showing that the numbers in the `QoG_pol` dataset is wrong (or at least its something odd here)

Then i found this site where i could download the data i wanted. At  <cite>[Stats.OECD.org][1]</cite>, I downloaded it as i couldn't find any other places where this data was accessible by url or through an api.

#### Its 80mb...

So i did some adjustments to the file before committing it to my public `data_folder` in the `class_datascience` repo on GitHub.It should be perfectly open for everyone, with all right reserved to OECD.
At first i downloaded the whole file locally. And then investigated what could be removed. I found out that there were maney categories for the environmental tax. All tax bases', 'Energy', 'Transport', 'Pollution', 'Resources. I'm interested in all tax bases, so that should shrank the data file a lot.

I'm leaving the code i used below, but commented out...

[1]:http://oe.cd/pine
[2]:https://www.compareyourcountry.org/environmental-taxes/en/1/183/default
[3]:https://stats.oecd.org/Index.aspx?DataSetCode=ERTR#

In [ ]:
# Importing the original dataset that QoG used, its huge...
#climate_tax = pd.read_csv('data/ERTR_29102022172652539.csv', low_memory=False)

# Checking the keys
#climate_tax.Category.unique()

# Narrowing
#climate_tax_total = climate_tax[climate_tax.Category == 'All tax bases']

# Exporting
#climate_tax_total.to_csv('oecd_climate_tax_total.csv')


That shrunk the filesize to 15mb, which is better. Ill uploead that on my public data_science repo, and then we can continue to invest


In [ ]:
url = 'https://raw.githubusercontent.com/percw/class_datascience/main/Notebooks/Assignment/individual_assignment/data/oecd_climate_tax_total.csv'
climate_tax = pd.read_csv(url, low_memory=False)

Now we need to do some cleaning and housekeeping...

In [ ]:
# We have a lot of combinations countrycode-year combinations. 
more_than_one_combination(climate_tax, 'COU', 'Year', 'Value')

# Here we can see what the keys looks like
climate_tax.columns

# I will check one country to start with to see if i can see that all of these values are about
switzerland = climate_tax[climate_tax.Country == 'Switzerland']
switzerland

# Looks like there are different categories. I want Climate change. Lets filter on that
switzerland_cc = switzerland[switzerland['Environmental domain'] == 'Climate change']
switzerland_cc

# And the variable im looking for is Tax revenue, % of GDP
switzerland_cc_tax = switzerland_cc[switzerland_cc['Variable'] == 'Tax revenue, % of GDP']

more_than_one_combination(switzerland_cc_tax, 'COU', 'Year', 'Value')
# After this we dont have any ambiguity in our country-year combination.


I have to look at the definition in the codebook given by QoG again.

> Climate change-related tax revenue as a percentage of gross domestic product (GDP). Includes
taxes, fees and charges, tradable permits, deposit-refund systems, subsidies, and voluntary approaches related to the domain of climate change.

It looks like QoG database used the `Tax revenue, % of total environmentally related tax revenue` value instead of `Tax revenue, % of GDP`. This value is much higher as it is the percentage of Tax related to climate change OVER environmental related tax revenue. Now it makes sense. Lets try to extract the correct values. I may be wrong here, overlooked something or misinterpreted the definition. But if not - i may have found an error in the QoG database.

So - to find the value that I want i need to define my subset with a couple of constraints.

- Environmental domain : Climate change
- Unit : Percentage
- Variable : Tax revenue, % of GDP
- CAT : TOT

In [ ]:

climate_tax = climate_tax[(climate_tax['Environmental domain'] == 'Climate change') & (climate_tax['Unit'] == 'Percentage') & (climate_tax['Variable'] == 'Tax revenue, % of GDP') & (climate_tax['CAT'] == 'TOT')]

# Great, it worked for the whole dataset as well.
more_than_one_combination(climate_tax, 'COU', 'Year', 'Value')


In [ ]:
# Now we can start exploring the new dataset and extracting what we want

ct_df = climate_tax[['COU', 'Year', 'Country', 'Value']]
ct_df.describe()


In [ ]:
# Having - % seems odd. Lets look a bit further

ct_df[ct_df.Value == -1.540000]

# Mexico seems to have several - values.
ct_df[ct_df.Country == 'Mexico']

# Lets see if any other countries have that as well
ct_df[ct_df.Value < 0]

# Its only mexico. I dont think these data are accurate so i may remove them...


In [ ]:

# ... before we remove them, i will only extract the countries we have in climate_challage_iso3_codes. 
# Maybe Mexico is not there. Which means that we do not have to do anything.

ct_df = ct_df[ct_df.COU.isin(climate_challage_iso3_codes)]
ct_df

In [ ]:
# Ok - so Mexico is still there, lets drop it.

mexico_indexes = ct_df[ct_df.Country == 'Mexico'].index

ct_df_clean = ct_df.drop(index=mexico_indexes)

In [ ]:
# Now i want to rename the columns to match the QoG data
ct_df_clean.rename(columns={'COU' : 'iso3', 'Year': 'year', 'Country' : 'country', 'Value' : 'oecd_cctr_gdp'}, inplace=True)
ct_df_clean

In [ ]:
# Now i want to see how dense my data is

ct_df_clean[pd.notnull(ct_df_clean['oecd_cctr_gdp'])].groupby(['year']).size().reset_index(name='counts')

In [ ]:
# We should see if there are any countries with very few tax registrations
df_oecd_cctr_gdp_count = ct_df_clean[pd.notnull(ct_df_clean['oecd_cctr_gdp'])].groupby(['country']).size().reset_index(name="counts")
df_oecd_cctr_gdp_count


In [ ]:

# These countries has less than 20 observations, i will remove thme 
countries = df_oecd_cctr_gdp_count[df_oecd_cctr_gdp_count.counts < 20].country.unique()
countries

In [ ]:

for c in countries:
	c_index = ct_df_clean[ct_df_clean.country == c].index
	ct_df_clean = ct_df_clean.drop(index=c_index)


In [ ]:
# Lets also make the oecd tax data sub subset from 2000 to 2018
climate_tax_df = ct_df_clean[(ct_df_clean.year >= 2000) & (ct_df_clean.year <= 2018)]
climate_tax_df

#### Lets merge and then aggregate

So we now have 3 different datasets.

- climate_tax_df : range 2000-2018. Climate Change tax % of GDP
- climate_challenge_clean : 2019. Score of Climate challenge a country faces
- df_co2_final : range 1750-2020. Total and per capita annual emission per country.

In [ ]:

tax_co2_df = pd.merge(left=df_co2_final, right=climate_tax_df, how='left', validate='1:1')
tax_co2_df

Here is the merged data (now called: `tax_co2_cc_df`) with the Climate Challenge Issue score from 2019.

In [ ]:
tax_co2_cc_df = pd.merge(left=tax_co2_df, right=climate_challenge_clean, how='left')
tax_co2_cc_df

### Question 11: Tasting and preparing the ingredients (univariate analysis) <a class="anchor" id="question11"></a>
Do an univariate analysis for each variable you have chosen (outcome variable, main explanatory variable and explanatory variable for heterogeneity):
- Prepare the variable, for example see if you need to transform the data further, i.e. log-transform, define a categorical variable, deal with outliers, etc.
- Understand the nature of the variable, i.e. continuous, categorical, binary, etc., which then allows to pick the right statistical tool in the bivariate analysis.
- Get an idea of the variable's behavior across time and space.

Describe these steps and the conclusions you can draw with the help of histograms, tables, maps and line graphs. **(3 points)**

In [ ]:
# Answer 11
# This Climate Change Tax is a variable i have investigated a lot, and i do not find any particular outliers. Its a continuous variable.
tax_co2_cc_df.oecd_cctr_gdp.describe()

In [ ]:
# This Co2 per capita is a variable i also have looked a lot into. The nature of it is continuous.
tax_co2_df.co2_per_capita.describe()
# Just checking for any anomalies or sudden changes.
tax_co2_df[tax_co2_df.co2_per_capita > 15]

In [ ]:
# To get a deeper understanding of the changes from year to year, we can use the method change() 
# to see the percentage change from year to year
# 
# This will catch any sudden and possibly wrong changes
# These rows represent the first year these countries registered co2
# as the definition of the change() function states that if the prior value is NaN
# So should the change be. I'm a bit puzzled why these aren't NaN...
# But i will keep these in mind when plotting.

df_co2_final.co2_per_capita.pct_change().describe()
df_co2_final[df_co2_final.co2_per_capita.pct_change() > 50]

In [ ]:
# Describing the values for Climate challenge Issues. Score with comma from 0-100.
climate_challenge_clean.describe()
climate_challenge_clean.epi_cch.unique()

#### To sum up so far and the variables i have:

- Climate Change Tax % of GDP (`oecd_cctr_gdp`) : continuous
- Climate Challenge (`epi_cch`) : continuous
- Co2 per capita (`co2_per_capita`) : continuous


### Now i want to visualize the three variables i have, to gain a general outlook before going into the bivariate analysis.

#### I will start by visualizing and exploring the data of `CO2 Emissions per capita`

In [ ]:
fig = px.scatter(tax_co2_df, y="co2_per_capita", x="year",
                 size="co2_per_capita", color="country", hover_name="country",
                 log_x=False)

fig.update_layout(
    title="CO2 emission per capita per country",
    xaxis_title="Year",
    yaxis_title="CO2 emission per capita (tons)",
    legend_title="Countries",
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="RebeccaPurple"
    )
)
fig.show()


I can see that Brunei has huge fluctuations. It has so in the source code as well, but i think these data-points will influence my results in a too strong way, so i will remove them.

In [ ]:
# Dropping Brunei

brunei_index = tax_co2_df[tax_co2_df.country == 'Brunei'].index
tax_co2_df.drop(index=brunei_index, inplace=True)
len(tax_co2_df[tax_co2_df.country == 'Brunei'])

In [ ]:
# Now - without Brunei

fig = px.scatter(tax_co2_df, y="co2_per_capita", x="year",
                 size="co2_per_capita", color="country", hover_name="country",
                 log_x=False)

fig.update_layout(
    title="CO2 emission per capita per country",
    xaxis_title="Year",
    yaxis_title="CO2 emission per capita (tons)",
    legend_title="Countries",
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="RebeccaPurple"
    )
)
fig.show()

In [ ]:
import plotly.express as px
fig = px.choropleth(tax_co2_df.sort_values('year', ascending=True), title='CO2 emission per capita per country per year',
                    locations="iso3", color="co2_per_capita", hover_name="country", animation_frame="year", range_color=[0.01, 10])
fig.show()


In [ ]:
fig = px.treemap(tax_co2_df, path=[px.Constant('world'), 'country'], values='co2_per_capita',
                 color='co2_per_capita', hover_data=['country'], title='Total CO2 Emissions per capita per country')
fig.update_layout(margin=dict(t=50, l=25, r=25, b=25))
fig.show()

In [ ]:
fig = px.scatter(climate_tax_df, y="oecd_cctr_gdp", x="year",
                 size='oecd_cctr_gdp', color="country", hover_name="country", range_x=[1999, 2020],
                 log_x=False)

fig.update_layout(
    title="Climate Change Tax % of GDP",
    xaxis_title="Year",
    yaxis_title="% of GDP",
    legend_title="Countries",
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="RebeccaPurple"
    )
)

fig.show()


In [ ]:
import plotly.express as px
fig = px.choropleth(climate_tax_df.sort_values('year', ascending=True), title='Climate Change Tax % of GDP',
                    locations='iso3', color='oecd_cctr_gdp', hover_name='country', animation_frame='year', range_color=[0.01, 5])
fig.show()


### Now i will move on to the Climate Change Issue data, and see how it looks like.

In [ ]:
fig = px.bar(climate_challenge_clean.sort_values('epi_cch', ascending=True), y="epi_cch", x='country',
             color="country", hover_name="country")

fig.update_layout(
    title="Climate Change Issue",
    showlegend=False,
    xaxis_title="Cuuntry",
    yaxis_title="Climate Change Issue Score",
    font=dict(
        family="Courier New, monospace",
        size=10,
        color="RebeccaPurple"
    )
)
fig.show()


### To log or not to log?

Now i will look into how symmetricly the data is distributed. We can do this by reviewing data skewness.

In [ ]:
# Simple function to address if skewness is in the range -3 to 3
def skew_within_range(s):
    if int(s) in range(-3, 3):
        return 'Yes'
    return 'No'


In [ ]:
# Starting with the CO2 data
co2_skew = skew(tax_co2_df["co2_per_capita"], nan_policy="omit")
c02_hist = px.histogram(data_frame=tax_co2_df, title="CO2 skewness: {:.2f}".format(co2_skew),
                        x='co2_per_capita', marginal="box", text_auto=True)
c02_hist.show()
print(f'Skew within range: {skew_within_range(co2_skew)}')

The Co2 data is heavily right skewed. But this is expected as are where fewer early recordings, and these were the "small" numbers. Recent and additionally is more dense data and of larger magnitude. I'm in the end going to aggregate this data, so i will not log transform it yet.

In [ ]:

# Continuing with the Climate Change Tax data
tax_skew = skew(tax_co2_df["oecd_cctr_gdp"], nan_policy="omit")
tax_hist = px.histogram(data_frame=tax_co2_df, title="Tax skewness: {:.2f}".format(tax_skew),
                        x='co2_per_capita', marginal="box", text_auto=True)
tax_hist.show()
print(f'Skew within range: {skew_within_range(tax_skew)}')


### The Climate Challenge Issue Category does not represent what i thought i did...

My initial interpretation of the `Climate Challenge Issue Score (epi_cch)` was that it represented the overall issues a country faces in regards to climate change. I got curious if this was correct after seeing the plot above. It did not make sense that Denmark (as one example) faces a lot of challenges - it may indeed have an overall low elevation above sea level. So i had to read the documentation again, and then again. The Climate Change Issue - is describing something completely different.

Definition:
> The Environmental Performance Index provides a ranking that shines light on how each country manages environmental issues. `The Climate Challenge Issue Category` (a sub-category of the EPI) consists of 8 indicators and how the country has dealt with these in the time range from approx 2001-2017. 

 I could have changed the value, pivoted the whole hypothesis and research question, but i think the mistake i made here represents how it is to deal with large datasets and complicated definitions. So i will keep transparency, incorporate my mistake - and take the implications it gives.

 This means, as variable for heterogeneity it does not work as well as intended. There will be some correlation between how much a country taxes climate change and its given score for how well it has handled climate change (represented by the 8 indicators in `epi_cch`). But i can use this variable to see if the tax is actually leading to effective measures taken to combat climate change! So in the sense, im deviating a bit - but the variable does give some more insight into the research question.

### Question 12: Cooking the ingredients together (bivariate analysis) <a class="anchor" id="question12"></a>

Considering the "nature" of your variables (continuous, categorical, binary, etc.), pick the right tool / tools for a preliminary bivariate analysis, i.e. correlation tables, bar/line graphs, scatter plots, etc. Use these tools to describe your preliminary bivariate analysis and your findings. **(3 points)**

Answer 12:

1. Firstly - I want to aggregate the `CO2 emissions per capita` (by sum/total) to only contain one value per country. As CO2 has an atmospheric lifetime estimated in the range between 5 and 1000 years (NASA: 300-1000, IPCC: 5-200) - CO2 emissions has a long-term effect!
2. Secondly - I want to take to aggregate `Climate change tax` by  averaging each countries values. I believe this gives the most accurate picture.
3. Thirdly - I will merge in `The Climate Change Issue`, the second explanatory variable.


In [ ]:
# Calculating the mean of climate change tax
tax_mean_df = tax_co2_df.groupby(['iso3', 'country'])['oecd_cctr_gdp'].mean().reset_index(name='oecd_cctr_gdp_mean')

# Calculating the sum of co2 emissions
co2_sum_df = tax_co2_df.groupby(['iso3', 'country'])['co2_per_capita'].sum().reset_index(name='co2_per_capita_total')

# Cleaning
tax_mean_df.dropna(inplace=True)

In [ ]:
# Merging tax_mean and co2_sum
tax_co2_agg_df = pd.merge(left=tax_mean_df, right=co2_sum_df, how='left', validate='1:1')
tax_co2_agg_df

# Merging tax_co2_agg and climate_challenge_issue
tax_co2_cc_df = pd.merge(left=tax_co2_agg_df, right=climate_challenge_clean, how='left', validate='1:1')
tax_co2_cc_df

# Cleaning
tax_co2_cc_df_clean = tax_co2_cc_df.copy()
tax_co2_cc_df_clean.dropna(inplace=True)
tax_co2_cc_df_clean

### Geography

As some countries were dropped, i think its important to again have a look at the geographical distribution of our aggregated data. The climate change tax variable, is derived from OECD - impacting exactly this scope.

In [ ]:
import plotly.express as px
fig = px.choropleth(tax_co2_cc_df_clean, title='Geographical distribution',
                    locations='iso3', color='country', hover_name='country')
fig.show()


We are missing United Kingdom, USA, Brazil and Russia (among others) - big polluters that may influence my findings. OECD countries are heavily represented - this may also influence my results as they (aim) to work towards a sustainable world.

### Again... To log or not to log

Now that i have aggregated the data, its time to investigate the skewness again. I found a plot i think illustrated how the data is distributed in a nice way. I also wrote a simple function checking if the skewness is within the range of -3 and 3.

In [ ]:
# Investigating skewness for aggregated data
# Total Co2 per capita
co2_skew = skew(tax_co2_cc_df_clean["co2_per_capita_total"], nan_policy="omit")
co2_hist = px.histogram(data_frame=tax_co2_cc_df_clean, title="Total CO2 per capita skewness: {:.2f}".format(co2_skew),
                        x='co2_per_capita_total', marginal="box", text_auto=True)
co2_hist.show()
print(f'Skew within range: {skew_within_range(co2_skew)}')

In [ ]:
# Mean Climate Change Tax
cctax_skew = skew(tax_co2_cc_df_clean["oecd_cctr_gdp_mean"], nan_policy="omit")
cctax_hist = px.histogram(data_frame=tax_co2_cc_df_clean, title="Mean Climate Change Tax skewness: {:.2f}".format(cctax_skew),
                        x='oecd_cctr_gdp_mean', marginal="box", text_auto=True)
cctax_hist.show()
print(f'Skew within range: {skew_within_range(cctax_skew)}')

In [ ]:
# Mean Climate Change Tax
epi_cch_skew = skew(tax_co2_cc_df_clean["epi_cch"], nan_policy="omit")
epi_cch_hist = px.histogram(data_frame=tax_co2_cc_df_clean, title="Climate Change Issue skewness: {:.2f}".format(epi_cch_skew),
                        x='epi_cch', marginal="box", text_auto=True)
epi_cch_hist.show()
print(f'Skew within range: {skew_within_range(epi_cch_skew)}')

### Lets visualize! 

Now that we have checked for skewness, its time to make some plots. The first plot shows my outcome variable : **Carbon Tax** vs the first explanatory variable : **Total CO2 emission**.

In [ ]:
fig = px.scatter(tax_co2_cc_df_clean.sort_values('co2_per_capita_total', ascending=True), y="oecd_cctr_gdp_mean", x="co2_per_capita_total",
                 size="co2_per_capita_total", color="country", hover_name="country", hover_data={'co2_per_capita_total': ':.1f', 'oecd_cctr_gdp_mean': ':.1f'},
                 labels={'oecd_cctr_gdp_mean': 'Mean CC Tax (% of GDP)',
                         'co2_per_capita_total': 'Total C02 Emission per capita'},
                 log_x=False)

fig.update_layout(
    title="Climate Tax of GDP vs Total C02 emissions per capita",
    xaxis_title="Total CO2 per capita (tons)",
    yaxis_title="Avg Climate Tax % of GDP(2000-2017)",
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="RebeccaPurple"
    )
)
fig.show()


My second plot, shows the same data but with a best fitting OLS line - highligting the positive correlation.

In [ ]:
fig = px.scatter(tax_co2_cc_df_clean.sort_values('co2_per_capita_total', ascending=True), y="oecd_cctr_gdp_mean", x="co2_per_capita_total",
                 size="co2_per_capita_total", color="oecd_cctr_gdp_mean", hover_name="country", trendline='ols', hover_data={'co2_per_capita_total': ':.1f', 'oecd_cctr_gdp_mean': ':.1f'},
                 labels={'oecd_cctr_gdp_mean': 'Mean CC Tax (% GDP)',
                         'co2_per_capita_total': 'Total C02 Emission per capita'},
                 log_x=False)

fig.update_layout(
    title="Climate Change Tax % of GDP vs Total C02 emissions per capita - with OLS trendline",
    xaxis_title="Total CO2 per capita (tons)",
    yaxis_title="Avg Climate Tax % of GDP(2000-2017)",
    font=dict(
        family="Courier New, monospace",
        size=10,
        color="RebeccaPurple"
    )
)
results = px.get_trendline_results(fig)
fig.show()


My third scatter plot, includes the second explanatory varaible - **Climate Change Issue** (from the Environmental Performance Index). Be observant of the new color shading.

In [ ]:
fig = px.scatter(tax_co2_cc_df_clean.sort_values('co2_per_capita_total', ascending=True), y="oecd_cctr_gdp_mean", x="co2_per_capita_total",
                 size="co2_per_capita_total", color="epi_cch", hover_name="country", trendline='ols', hover_data={'co2_per_capita_total': ':.1f', 'oecd_cctr_gdp_mean': ':.1f'},
                 labels={'oecd_cctr_gdp_mean': 'Mean CC Tax (% GDP)',
                         'co2_per_capita_total': 'Total C02 Emission per capita'},
                 log_x=False)

fig.update_layout(
    title="Climate Change Tax % of GDP vs Total C02 emissions per capita - with OLS trendline",
    xaxis_title="Total CO2 per capita (tons)",
    yaxis_title="Avg Climate Tax % of GDP(2000-2017)",
    font=dict(
        family="Courier New, monospace",
        size=10,
        color="RebeccaPurple"
    )
)
results = px.get_trendline_results(fig)
fig.show()


We can see that the darker/purple colors are usually located below the line. And the light orange/yellow colors are above the line. What does this mean? As mentioned earlier my second explanatory variable may not have been working so well for heterogeneity, as i misinterpreted its definition. But it may give an indication on wether a country is successful at turning climate change tax into effective measures. Either way - i will investigate the correlation between my outcome variable : **Climate Change Tax** and the second explanatory variable : **Climate Change Issue**.

In [ ]:
tax_co2_cc_df_clean[['oecd_cctr_gdp_mean', 'epi_cch']].corr()

The correlation of `0.65` is quite strong - the same impression i got from looking at the scatter plot. So how well a country performs on the climate change issue is a strong predictor of how much a country taxes climate change. Or is it the way around?... They both describe the same scope, so it is natural that they are tightly coupled. This is what i expected - but again not my initial intention. We can visualize the relationship clearer with the scatterplot below.

In [ ]:
fig = px.scatter(tax_co2_cc_df_clean, y="oecd_cctr_gdp_mean", x="epi_cch",
                 size="oecd_cctr_gdp_mean", color="epi_cch", hover_name="country", trendline='ols', hover_data={'epi_cch': ':.1f', 'oecd_cctr_gdp_mean': ':.1f'},
                 labels={'oecd_cctr_gdp_mean': 'Mean CC Tax (% GDP)',
                         'epi_cch': 'Climate Change Issue'},
                 log_x=False)

fig.update_layout(
    title="Climate Change Tax % of GDP vs Climate Change Issue - with OLS trendline",
    xaxis_title="Climate Change Issue (EPI score)",
    yaxis_title="Avg Climate Tax % of GDP(2000-2017)",
    font=dict(
        family="Courier New, monospace",
        size=10,
        color="RebeccaPurple"
    )
)
fig.show()


In [ ]:

print(results.px_fit_results.iloc[0].summary())

### Question 13: Tasting the new recipe (conclusion) <a class="anchor" id="question13"></a>

Explain what you learned, the problem faced, what would you do next (you can suggest other data you would like to have etc). **(2 points)**

Answer 13:

### Learnings and Problems

My second explanatory variable did not impose the heterogeneity I wanted. This was due to documentation misreading. I acknowledge that I'm not the first to do so. Evidently, through the error I found in the QOG database, they had misunderstood the epi_cch score. They used `Tax revenue, % of total environmentally related tax revenue` instead of `Climate change-related tax revenue as % of GDP`. I have learned that no perfect dataset exists. If I wanted to stay with my hypothesis and the variables I had chosen, I needed to get the correct data. I did and quickly understood why the QoG researchers were wrong. This was a massive dataset (> 70MB) with many subcategories in countless domains. Luckily I found what I was searching for and could continue.

Some of the problems, issues, and learnings I've met and gained during this exercise were:

- It is hard to work with data from different years in one dataframe where NaNs are expected
- Interpreting definitions can cause problems
- Keeping track of all the columns and their respective names is hard!
- Naming conventions are fundamental. I need to be more consistent at this. E.g., always use df first vs. last in a dataframe declaration. Note to self: all "final" dataframes should end with `_final`
- Cleaning data takes time
- Finding the perfect variable for your hypothesis is very hard (impossible?)
- Manipulating a copy of the "real" dataframe before doing it on the original is vital
- Splitting coding cells is efficient and saves a lot time
- It's possible to spend an endless amount of time making plots pretty

### Reflections and going further

The general observation from the plots is that a positive correlation exists between the **total CO2 emissions per capita** and the **tax mean related to climate change in % of GDP** with a coefficient of 1.089. The statistical test done above emphasizes that 13.6% of the variance of the climate change tax (% of GDP) is explained by the variance of total Co2 emission. (R^2 of 0.136). Furthermore, it highlights a very low P value (<0) and an F-stat at 11.31. I will not go further into these statistical tests but focus on my interpretation of the data and plausible turns I could have taken. 

What does my outcome tell me? First of all - there may be several drivers explaining the positive correlation. Countries with high total CO2 emissions (FYI: intuitive reflection incoming) are usually countries with a lot of trade (i could look further into this) - that given, it is vital to have a common value-set to cooperate efficiently and with respect. And as climate change is a global problem, countries that neglect climate change could face difficulties trading proficiently  - and we could therefore see that this effect could collude with the relationship I found. A solution to this could be to use `% of GDP coming from trade` as a second or third explanatory variable.

I introduced this exploratory part with the term climate responsibility. Can we call these countries climate responsible because of the positive correlation? No. I don't think so. First of all, taxes do not directly translate into being responsible. What does the taxes do? Are they effective at meeting climate change? My findings do not say anything about this. Additionally, being from Norway (with a total of 670 tons co2 per capita), rich oil countries are entirely independent of the oil they dig up and sell in statistics as these. Does to not burn to mean to not be responsible, even though you dug it and sold it?

Another fundamental notion is that I could have chosen another explanatory value when I realized my wrongdoing. After studying the `QoG Codebook,` again, the `Total damage from natural disasters in USD (emdat_damage)` could have made a good variable for heterogeneity. A country facing many climate threats will consequently spend much money repairing damages from incidents, s.a., extreme weather. This variable describes the cost of natural disasters – and, therefore, indirectly, the burden of climate change. Lastly, as the USA, Great Britain, and Russia are some of the biggest polluters, I would use an outcome variable that includes these countries (but I wanted to avoid using the same value as Quentin). 


Thank you for making such a comprehensive, challenging, and fun assignment - I learned a lot!

To shwocase and motivate my own progression with data science i made a portfolio hosted og GitHub. If you want, you can check it out at https://percw.github.io/. Its a work in progress, currently working on making it interactive with Plotply Dash.


### Computing Environment

In [ ]:
%load_ext watermark
%watermark -v -p jupyterlab